In [1]:
import pandas as pd 
import numpy as np
import re
import copy

In [4]:
def processing_itp(filename="martini_v2.P_supp-material.itp"):
    '''
    Takes a itp file and
    Returns a dictionary with keys as directives and values as directive_contents
    '''
    with open(f"{filename}") as f:
        params = f.readlines()

    # First, remove all comment lines
    params = [m for m in params if m[0] != ';']
    # Second, remove all empty lines, i.e., 
    # when stripped of white spaces and newline characters, lenght of the string ==0
    params = [m.strip() for m in params if len(m.strip()) > 0]
    
    # First, we find the indices of the starting points of each directive.
    # We can identify the starting points by looking for lines beginning with '['

    directive_start_positions = [ [line, current_ind]          # each element contains directive_name the corresponding index
        for current_ind, line in enumerate(params) # parse each line keeping track of the current index
        if re.search( '\[.+', line)]                       
    # .+ operator is equibalent to '*' in regex. '[' is a special character in regex, so \ should be used to find the literal '['

    # The actual content of each directive starts after the '\[.+' line, and ends before the next '\[.+' line
    # Now we will build a dictionary where directive_names are keys, and directive contents are values.
    directive_contents = {}
    i = 0
    while i< len(directive_start_positions):
        directive_name, start_position = directive_start_positions[i]
        directive_name = directive_name.strip('[').strip(']').strip() # remove extra characters
        # print(directive_name)
        if i == len(directive_start_positions)-1: # special case of last element
            directive_contents[directive_name] = params[start_position+1:] # load content till the end of the file
        else: 
            next_directive_start = directive_start_positions[i+1][1]
            # print(start_position, next_directive_start)
            directive_contents[directive_name] = params[start_position+1:next_directive_start]
        i+=1

    directive_contents.keys()
    return directive_contents

###########################################################################################################

def unique_beads(filename="popc.itp"):
    '''
    Takes in a molecule.itp file and 
    returns a list of unique beadtypes required to be defined for it. 
    '''
    # get the directive_contents dictionary 
    directive_contents = processing_itp(filename=filename)
    beads = [i.split()[1] for i in directive_contents['atoms']]
    
    return set(beads)


###############################################################
'''
Getting the desired beads.
'''
beads = [];
# Making a list of beadtypes
for file in ['popc.itp', '../no_mem/abeta1.itp', '../350/0.136/trial1.itp']:
    beads += unique_beads(filename=file)
    
for bead in beads:
    if len(bead) > 5:
        index = beads.index(bead)
        beads[index] = bead[:2]
    
print(set(beads))
print(len(set(beads)))
desired_beads = set(beads)
# Discarding the DUM non-interacting bead present in cholesterol 
# desired_beads.remove('DUM')
# desired_beads.remove('Dvs')
desired_beads.remove('D1')
desired_beads.remove('D2')

{'AR', 'Qd', 'D2', 'Qa', 'SC3', 'Q0', 'D1', 'C1', 'SP1', 'DUM', 'PNa', 'PP5', 'C3', 'SC1'}
14


In [5]:
copy_desired_beads = copy.deepcopy(desired_beads)
for b in copy_desired_beads:
    if b[-2:] == '_L':
        desired_beads.discard(b)

#Removing the _P out of protein beads introduced because of repulsion on lower leaflet and WC1
for b in copy_desired_beads:
    if b[-2:] == '_P':
        desired_beads.add(b[:-2])
        desired_beads.discard(b)
    if b =='WC1':
        desired_beads.discard(b)
print(desired_beads)

{'AR', 'Qd', 'Qa', 'SC3', 'Q0', 'C1', 'SP1', 'DUM', 'PNa', 'PP5', 'C3', 'SC1'}


In [6]:
def get_beads_in_martini(file="martini_v2.P_supp-material.itp"):
    '''
    Returns a list of beads present in the martini ff.
    '''
    directive_contents = processing_itp(filename=file)
    small_beads = []
    regular_beads = []
    special_cases = []
    for atom in directive_contents['atomtypes']:
        atom_name = atom.split()[0]
        if atom_name[0] == 'S':
            small_beads.append(atom_name)
        elif atom_name[0] in ['P', 'N', 'C', 'Q']:
            regular_beads.append(atom_name)
        else: # most likely if atom is D
            special_cases.append(atom_name)
    beads_in_martini = [bead for li in [regular_beads, small_beads, special_cases] for bead in li]
    
    print(len(beads_in_martini))
    
    return beads_in_martini

original_martini_beads = get_beads_in_martini(file="martini_v2.P_supp-material.itp")

38


In [7]:
def get_interaction_data(file="martini_v2.P_supp-material.itp"):
    '''
    Returns a dictionary with beads as keys and sigma & epsilon as 
    values.
    '''
    directive_contents = processing_itp(filename=file)   
    nb_param_copy = copy.deepcopy(directive_contents['nonbond_params'])
    for l_index, l in enumerate(nb_param_copy):
        l_processed= l.split()

    # Now l_processed contains: atom_i, atom_j, funct, c6, c12, ... comments
    # we will keep the first 5 elements, and discard excess elements
        nb_param_copy[l_index] = l_processed[:5]
    
    #grab atom_i-->atom_j pairs
    ij_pairs = [tuple(nbpar[0:2]) for nbpar in nb_param_copy] 
    # get corresponding c6 and c12 terms
    c6 = np.array([float(nbpar[3]) for nbpar in nb_param_copy])
    c12 =  np.array([float(nbpar[4]) for nbpar in nb_param_copy])
    # convert c6 c12 to sigmas and epsilons
    sigmas_list = np.round((c12/c6)**(1/6), decimals=2)
    epsilons_list = np.round((c6**2)/(4*c12), decimals=2)

    # Interactions dictionary

    martini_int_dict = {ij_pairs[i]: {
                    'sigma': sigmas_list[i],
                    'epsilon' : epsilons_list[i],
                    } 
                    for i in range(len(ij_pairs))
                }
    
    return martini_int_dict

martini_int_dict = get_interaction_data(file="martini_v2.P_supp-material.itp")

/tmp/ipykernel_5470/3198637920.py:21: RuntimeWarning: invalid value encountered in divide
  sigmas_list = np.round((c12/c6)**(1/6), decimals=2)
/tmp/ipykernel_5470/3198637920.py:22: RuntimeWarning: invalid value encountered in divide
  epsilons_list = np.round((c6**2)/(4*c12), decimals=2)


In [8]:
'''
These bead types are considered similar to the martini interactions

similar = ['POL', 'C1', 'C2', 'C3', 'C4', 'C5', 
            'SC1', 'SC2', 'SC3', 'SC4', 'SC5', 'Qa', 
            'Q0', 'Qd', 'N0', 'Nd', 'Nda', 'SNd', 'SN0']
'''
# beads to be polarized from ProMPT 

def add_beads_without_cation_pi (pep_seq = "KLVFFAE", n_peptides = 16):
    '''
    Returns two lists: one for the new beads (in ProMPT) 
    and the other is a reference list with beads from the martini ff on which these
    new beads are based.
    '''
    polar_aromatic_resids = [resid for resid, resname in enumerate(pep_seq) if resname in 'YWH']
    apolar_aromatic_resids = [resid for resid, resname in enumerate(pep_seq) if resname =='F']
    cationic_resids = [resid for resid, resname in enumerate(pep_seq) if resname in 'KR']
    aromatic_resids = polar_aromatic_resids + apolar_aromatic_resids
    
    #Add non-aromatic polarized beads. 
    #Polarized beads of polar type are differentiated by a 'P' prefix
    to_be_polarized = ['P5', 'P4', 'P1', 'Na', 'Nda', 'N0', 'SP1', 'SNd'] 
    polarized_beads = ['P'+bead for bead in to_be_polarized]
    
    #Add polarized aromatic beads (ARP)
    #Aromatic beads are based on the MARTINI SC1 particle
    polarized_beads += ['ARP'] 
    to_be_polarized += [ 'SC1']
    
    #Add regular aromatic beads
    new_aromatic_beads = ['AR'] 
    aromatic_reference_beads = ['SC1']
    
    #Beads for proline
    polarized_beads.append('APP5')
    to_be_polarized.append('P5')
    ## Add PAC1, a bead similar to C1, but special for Proline
    new_aromatic_beads += ['PAC1']
    aromatic_reference_beads += ['C1']
                          
    newly_added_beads = polarized_beads + new_aromatic_beads 
    reference_beads = to_be_polarized + aromatic_reference_beads 
    
    return newly_added_beads, reference_beads
    

In [9]:
newly_added_beads, reference_beads = add_beads_without_cation_pi(pep_seq = "KLVFFAE", n_peptides = 1)

In [10]:
def adding_new_interactions(martini_int_dict = martini_int_dict, new_beads = newly_added_beads, ref_beads = reference_beads):
    '''
    Adding interactions for the new beads to the martini dictionary.
    Returning the dictionary with the new interactions.
    '''
    #Make a working copy of martini_int_dict
    int_dict = copy.deepcopy(martini_int_dict)
    
    #New interactions will be stored in this list
    new_interactions = set([])
    
    #Adding self-term for new beads.
    for ref, new in zip(ref_beads, new_beads):
        int_dict[(new, new)] = int_dict[(ref, ref)]
        new_interactions.add((new,new))
        
    #Add new_bead---new_bead interactions
    i=0;
    while i < len(new_beads):
        new_i = new_beads[i]
        ref_i = ref_beads[i]

        j = i+1 # skip self-interactions

        while j < len(new_beads):
            new_j = new_beads[j]
            ref_j = ref_beads[j]

            ref_pair = (ref_i, ref_j)
            new_pair = (new_i, new_j)
            if ref_pair in int_dict: # check if ref_pair is a key in int_dict
                int_dict[new_pair] = int_dict[ref_pair]
                new_interactions.add(new_pair)

            elif ref_pair[::-1] in int_dict: # check if ref_pair is a key, but in reverse order, in int_dict
                int_dict[new_pair[::-1]] = int_dict[ref_pair[::-1]]
                new_interactions.add(new_pair[::-1])
            j+=1
        i+=1
        
    #Add cross-terms for new_beads--original_bead interactions
    #Values of cross terms are identical to those involving unpolarized beads
    original_martini_beads = get_beads_in_martini(file="martini_v2.P_supp-material.itp")
    i=0;
    while i< len(new_beads):
        new_i = new_beads[i]
        ref_i = ref_beads[i]
        for org_j in original_martini_beads:   
            ref_pair = (ref_i, org_j)
            new_pair = (new_i, org_j)
            if ref_pair in int_dict.keys():
                int_dict[new_pair] = int_dict[ref_pair]
                new_interactions.add(new_pair)
            elif ref_pair[::-1] in int_dict.keys():
                int_dict[new_pair[::-1]] = int_dict[ref_pair[::-1]]
                new_interactions.add(new_pair[::-1])
        i+=1
    
    return int_dict, new_interactions

In [11]:
int_dict, new_int = adding_new_interactions(martini_int_dict = martini_int_dict, new_beads = newly_added_beads, 
                                            ref_beads = reference_beads)
interaction_data_frame = int_dict;
new_bead = 'POL';
reference_bead='POL';
def add_redundant_bead(interaction_data_frame, new_bead, reference_bead):

    """
    Usage:
    int_df = pd.DataFrame.from_dict(int_dict, orient='index')
    #### Add PAC1
    with_pac1 = add_redundant_bead(int_df, 'PAC1', 'C1')
    """
    int_df = interaction_data_frame.copy()
    new_bead=new_bead
    reference_bead=reference_bead

    for k0, k1 in int_df.index:
        
        # Add self-interaction
        if all([k0 == reference_bead, k1==reference_bead]):
            new_k0, new_k1 = new_bead, new_bead
            
        elif k0 ==reference_bead:
            new_k0, new_k1 = new_bead, k1
            
        elif k1 == reference_bead:
            new_k0, new_k1 = k0, new_bead
            
        else:
            continue
        
        ndf = pd.DataFrame(
            {
                "sigma": [int_df.loc[k0, k1]['sigma']],
                "epsilon": [int_df.loc[k0, k1]['epsilon']],
            }, 
            index=[(new_k0, new_k1)]
        )

        int_df = pd.concat([int_df, ndf])

    return int_df

38


In [12]:
 int_df = pd.DataFrame.from_dict(int_dict, orient='index')

In [13]:
def changing_old_intn_without_cation_pi(interaction_dict=int_dict, new_interactions = new_int):
    '''
    Making the edits in the epsilon values according to 
    the ProMPT paper.
    '''
    # Store int_dict as a dataframe. 
    int_df = pd.DataFrame.from_dict(interaction_dict, orient='index')
    
    """
    1. Change C1--POL, SC1--POL, PAC1--POL, AR*--POL, ARP*--POL interactions
    This makes these hydophobic beads more averse to water
    """

    # PAC1 and C1 are the same basically
    # Naming of PAC1 differs to incorporate cation-pi and pi-pi interactions
    int_df.loc['POL', 'C1']['epsilon'] = 1.0    
    int_df.loc['POL', 'PAC1']['epsilon'] = 1.0
    
    # I don't know why POL--SC1 is different from POL--AR
    # However, SC1 is not used in the proteins, only AR is.
    int_df.loc['POL', 'SC1']['epsilon'] = 1.0 
    int_df.loc['POL', 'AR']['epsilon'] = 1.5
    
    # Note this is corrected again, since ARP and POL are both polarized beads.
    int_df.loc['POL', 'ARP']['epsilon'] = 1.5  # Originally 1.9
    
    """
    2 Make corrections to polarized-polarized interactions
    """
    new_beads = add_beads_without_cation_pi(pep_seq = "KLVFFAE", n_peptides = 16)[0]
    polarized_beads = [ bead for bead in new_beads if bead[0]== 'P' and bead != 'PAC1'] + ['ARP', 'APP5']
    print(polarized_beads)
    polarizable_correction_factor = 0.58 # epsilon correction in kJ/mol. 
    #Comes from Prompt. however basis unknown.
    ## parse the list of all new interactions
    ## if the interaction involves 2 polarizable beads,
    ## Reduce their epsilon by the correction factor
    # correction_factor = 0.58
    count=0
    for (i, j) in list(new_interactions):
        if all(bead in polarized_beads for bead in [i, j]): #iterating over the beads in the tuple

            old_eps = int_df.loc[i,j]['epsilon']
            # print('que', i, j)
            int_df.loc[i,j]['epsilon'] -= polarizable_correction_factor
            new_eps = int_df.loc[i,j]['epsilon']
#             list_of_polarized_polarized_interactions[(i,j)] = [f"{old_eps:.3f}", f"{new_eps:.2f}"]
            count+=1

    """
    3. POL-polarized beads water adjustments
    """
    
    polar_to_water_correction_factor = 0.58
    count=0
    for (i, j) in list(new_interactions):
        if 'POL' in [i,j]:
            if any(bead in polarized_beads for bead in [i, j]):
                old_eps = int_df.loc[i,j]['epsilon']
                int_df.loc[i,j]['epsilon'] -= polar_to_water_correction_factor
                new_eps = int_df.loc[i,j]['epsilon']
#                 list_of_polarized_POL_interactions[(i,j)] = [f"{old_eps:.3f}", f"{new_eps:.2f}"]
                count+=1
    
    
    """
    4. Charged-polarized edits
    """
    charged_beads = [k for k in original_martini_beads if 'Q' in k]
    q_correction_factor = 0.28
    count=0
    for (i, j) in list(new_interactions):
        ## check if one charged bead and one pol bead are present in (i,j)
        if all(
                [
                    any(qbead in charged_beads for qbead in [i,j]),
                    any(pbead in polarized_beads for pbead in [i, j])
                    ]
                ):
                old_eps = int_df.loc[i,j]['epsilon']
                int_df.loc[i,j]['epsilon'] -= q_correction_factor
                new_eps = int_df.loc[i,j]['epsilon']
#                 list_of_polarized_charged_interactions[(i,j)] = [f"{old_eps:.3f}", f"{new_eps:.2f}"]
                count+=1

    """
    5. Dummies
    """
    ## POL - dummy interactions are not present in martini_v2.P.
    # Adding them here
    pol_d = pd.DataFrame({"sigma": np.inf,
                                "epsilon": np.inf,
                                }, index=[('POL', 'D')])
    int_df = pd.concat((int_df, pol_d))

    ## Add new_dummies---D
    #D1 dummies belong to lipids and have charge +/- 0.175
    #D2 dummies belong to protein backbone & carry +/- 0.34
    #D2A are dummies present on sidechains of aromatic residues
    new_dummies = ['D1', 'D2', 'D2A'] # DT is dummy for tiny
    #Dvs is for CHOL 
    for new_d in new_dummies:
        int_df = add_redundant_bead(int_df, new_d, 'D')
        new_d_interaction = pd.DataFrame({"sigma": np.inf,
                                "epsilon": np.inf,
                                }, index=[(new_d, 'D')])
        int_df = pd.concat((int_df,new_d_interaction))
        
        
    """
    6. PNa alterations
    """
    lip_beads_CP = {'Q0':3.2, 'Qa':3.6, 'PP5':4.7, 'PNa':3.2}
    alt_bead = ['PNa']
    for i in alt_bead:
        for j in lip_beads_CP.keys():
            try:
                old_eps = int_df.loc[i,j]['epsilon']
                int_df.loc[i,j]['epsilon'] = lip_beads_CP[j]
                new_eps = int_df.loc[i,j]['epsilon']
                print(i, j, old_eps, new_eps)
            except:
                old_eps = int_df.loc[j,i]['epsilon']            
                int_df.loc[j, i]['epsilon'] = lip_beads_CP[j]
                new_eps = int_df.loc[j, i]['epsilon']
                print(i, j, old_eps, new_eps)
    
    return int_df

In [32]:
# lip_beads_CP = {'Q0':3.2, 'Qa':3.6, 'PP5':4.7, 'PNa':3.2}
# alt_bead = ['PNa']
# for i in alt_bead:
#     for j in lip_beads_CP.keys():
#         try:
#             old_eps = int_df.loc[i,j]['epsilon']
#             int_df.loc[i,j]['epsilon'] = lip_beads_CP[j]
#             new_eps = int_df.loc[i,j]['epsilon']
#             print(i, j, old_eps, new_eps)
#         except:
#             old_eps = int_df.loc[j,i]['epsilon']            
#             int_df.loc[j, i]['epsilon'] = lip_beads_CP[j]
#             new_eps = int_df.loc[j, i]['epsilon']
#             print(i, j, old_eps, new_eps)

In [14]:
int_df = changing_old_intn_without_cation_pi(interaction_dict=int_dict, new_interactions = new_int)
def convert_to_c6_c12(int_df=int_df):
    converted_df = pd.DataFrame({'c6': [None]*len(int_df),
                                    'c12': [None]*len(int_df),
                                    'sigma': [f'; {sig:.3f}' for sig in int_df['sigma']],
                                    'epsilon': [f'; {eps:.3f}' for eps in int_df['epsilon']],
                                    },
                                    index=int_df.index
                                    )

    for ind in converted_df.index:
        sig = int_df.loc[ind]['sigma']
        eps = int_df.loc[ind]['epsilon']
        c6 = 4*eps*(sig**6)
        c12 = 4*eps*(sig**12)
        sig = f'; {sig:.3f}'
        eps = f'; {eps:.3f}'
        converted_df.loc[ind] = [c6, c12, sig, eps]
        # sigma, epsilon = int_df.loc[k0, k1]
        # converted_df.loc[k0, k1]['c6'] = 4*epsilon*(sigma**6)
        # converted_df.loc[k0, k1]['c12'] = 4*epsilon*(sigma**12)

    # All D -- any bead interactions should be 0
    for ind in converted_df.index:
        if any([bead=='D' for bead in ind]):
            sig = '; no LJ interaction'
            eps = ''
            converted_df.loc[ind] = [0, 0, sig, eps]
            
    # Any interaction with the new dummies shoudld have a hardcore repulsion factor
    new_dummies  = ['D1', 'D2', 'D1A', 'D2A', 'Dvs']
    for ind in converted_df.index:
        if any([bead in new_dummies for bead in ind]):
            sig = '; sigma infinity'
            eps = ''
            converted_df.loc[ind] = [0, 4.5387e-10, sig, eps]

    return converted_df

x = convert_to_c6_c12(int_df)


['PP5', 'PP4', 'PP1', 'PNa', 'PNda', 'PN0', 'PSP1', 'PSNd', 'ARP', 'APP5']
PNa Q0 4.22 3.2
PNa Qa 4.22 3.6
PNa PP5 4.42 4.7
PNa PNa 3.42 3.2


In [15]:
def gen_x_without_cation_pi(file="martini_v2.P_supp-material.itp", pep_seq  = 'KLVFFAE'):
    '''
    Generating the forcefield file without cation-pi.
    '''    
    #Get the beads
    newly_added_beads, reference_beads = add_beads_without_cation_pi(pep_seq = "KLVFFAE", n_peptides = 16)
    
    #Add new interactions
    martini_int_dict = get_interaction_data(file="martini_v2.P_supp-material.itp")
    int_dict, new_interactions = adding_new_interactions(martini_int_dict = martini_int_dict, new_beads = newly_added_beads, ref_beads = reference_beads)
    
    #Making edits
    int_df = changing_old_intn_without_cation_pi(interaction_dict=int_dict, new_interactions = new_interactions)
    
    #Getting x 
    x = convert_to_c6_c12(int_df)
    
    return x

In [1]:
x = gen_x_without_cation_pi(file="martini_v2.P_supp-material.itp", pep_seq  = 'KLVFFAE')
def write_ff_without_cation_pi(file = "martini_v2.P_supp-material.itp", ff_file="test"):
    '''
    #Assembling a forcefield.
    '''
    #Get the directive_contents 
    directive_contents=processing_itp(filename=file)
    
    #Writing file
    defaults_directive = ['[ defaults ]', "1 1 no 0.0 0.0", '']
    
    """
    Write atomtypes directive

    classify all beads into following categories:
    >>> martini_2P_beads        (from polarizable water paper)
    >>> new_polarized_beads     
    >>> new_dummies         
    >>> new_non_polarized_beads
    """
    ## First, all the stanbdard martini_beads
    #map(fun, iter):returns a list of the results after applying the given 
    #function to each item of the iterable
    martini_2P_beads = list(map(lambda x: x.split()[0],
                                directive_contents['atomtypes'])
                                )
    ## Now parse the newly made non_bonded parameters list to identify new beads
    all_beads = []
    for ind in x.index:
        all_beads += list(ind)
    all_beads = pd.unique(all_beads)
    new_beads = [bead for bead in all_beads 
                    if bead not in martini_2P_beads
                    ]
    ## now to categorize the new_beadtypes
    polarized_beads = [ bead for bead in newly_added_beads if bead[0]== 'P' and bead != 'PAC1']
    new_dummies  = ['D1', 'D2', 'D1A', 'D2A']
    new_polarized_beads = polarized_beads # previously defined
    new_dummies = new_dummies             # previously defined
    new_non_polarized_beads = [bead for bead in new_beads 
                                if all([bead not in polarized_beads,
                                        bead not in new_dummies])
                                ]

    """
    Now, create an atomtypes directive in GROMACS format
    Mass of all standard/unpolarized regular beads = 72 amu
    Mass of all standard/unpolarized small and tiny beads = 45 amu
    In polarized beads, total mass will be distributed across central, dummy+, and dummy- particles
    i.e., PP*: 72/3; PS*: 45/3; PT*: 45/3 
    """
    def atomtype_line(bead_name, mass):
        return f'{bead_name:<11} {mass} 0.00   A   0.0   0.0'

    atomtypes_directive = ['[ atomtypes ]', '; name mass charge ptype c6 c12']

    # add all martini_types
    atomtypes_directive += ['; Beads from MARTINI 2.P nucleic acid extension']
    for bead in martini_2P_beads:
        if bead[0] in ['S']: # list of tiny and small polarized beads
            bead_mass = 45.0
        elif bead in ['POL', 'D']:
            bead_mass = 24.0
        else:
            bead_mass = 72.0
        atomtypes_directive += [atomtype_line(bead, bead_mass)]

    # add all new polarized beads
    atomtypes_directive += ['; Central particles of polarized beads with internal dipoles']
    for bead in new_polarized_beads:
        if bead in [ 'PSNd','APC1']: # list of tiny and small polarized beads
            bead_mass = 15.0
        elif bead[:3] == 'ARP': # list of tiny and small polarized beads
            bead_mass = 15.0
        else:
            bead_mass = 24.0
        atomtypes_directive += [atomtype_line(bead, bead_mass)]

    # add new dummies
    atomtypes_directive += ['; Dummy particles of polarized beads with internal dipoles']
    for bead in new_dummies:
        if bead in ['D2A']: # list of dummies for tiny and small polarized beads
            bead_mass = 15.0
        else:
            bead_mass = 24.0
        atomtypes_directive += [atomtype_line(bead, bead_mass)]

    # add new non-polarized beads
    atomtypes_directive += ['; Newly defined non-polarized beads']
    for bead in new_non_polarized_beads:
        if bead[:2] == 'AR': # list of tiny and small non-polarized beads
            bead_mass = 42.0
        else:
            bead_mass = 72.0
        atomtypes_directive += [atomtype_line(bead, bead_mass)]
    atomtypes_directive += ['']

    """
    write nonbonded params directive
    """
    nb_params_directive = ['[ nonbond_params ]', '; i j funct c6 c12; sigma epsilon']

    def write_nbparam_line(ij_pair, c6_c12_df):
        i, j = ij_pair
        funct = 1
        c6, c12, sig, eps = c6_c12_df.loc[ij_pair]
        return f'{i:<11}\t{j:<11}\t {funct} {c6:.3E} {c12:.3E} {sig}{eps}'

    for ind in x.index:
        # print(write_nbparam_line(ind, x))
        nb_params_directive += [write_nbparam_line(ind, x)]

    nb_params_directive += ['']
    
    """
    Print to file
    """
    with open(f'{ff_file}.itp', 'w') as f:
        outdata = '\n'.join(defaults_directive+atomtypes_directive+nb_params_directive)
        f.write(outdata)

NameError: name 'gen_x_without_cation_pi' is not defined

In [17]:
write_ff_without_cation_pi(file = "martini_v2.P_supp-material.itp", ff_file="ff_pcchol_abeta.itp")

In [18]:
x

,,c6,c12,sigma,epsilon
P5,P5,0.241454,0.002603,; 0.470,; 5.600
SP5,SP5,0.106199,0.000671,; 0.430,; 4.200
P4,P4,0.215584,0.002324,; 0.470,; 5.000
SP4,SP4,0.09482,0.000599,; 0.430,; 3.750
P3,P3,0.215584,0.002324,; 0.470,; 5.000
...,...,...,...,...,...
D2A,PAC1,0,0.0,; sigma infinity,
POL,D2A,0,0.0,; sigma infinity,
D1,D2A,0,0.0,; sigma infinity,
D2,D2A,0,0.0,; sigma infinity,


In [22]:
'''
Cell to check the ff parameters. Given by Suhas on 25th Jan, 2024.
'''
list_of_beads = list(desired_beads)
list_of_beads.remove('DUM')
x = gen_x_without_cation_pi(file="martini_v2.P_supp-material.itp", pep_seq  = 'KLVFFAE')
eps_to_print_as_table = pd.DataFrame({bead: pd.Series([]) for bead in list_of_beads})
sig_to_print_as_table = pd.DataFrame({bead: pd.Series([]) for bead in list_of_beads})
print(list_of_beads)
for bead_i in list_of_beads:
    for bead_j in list_of_beads:

        try:
            sigma = x.loc[(bead_i, bead_j)]['sigma']
            epsilon = x.loc[(bead_i, bead_j)]['epsilon']
        except KeyError:
            sigma = x.loc[(bead_j, bead_i)]['sigma']
            epsilon = x.loc[(bead_j, bead_i)]['epsilon']
        # print(f"{bead_i}-{bead_j} {epsilon.strip(';').strip('')}")
        eps = f"{epsilon.strip(';').strip()}"
        sig = f"{sigma.strip(';').strip()}"
        eps_to_print_as_table.at[bead_i,bead_j] = eps
        sig_to_print_as_table.at[bead_i,bead_j] = sig

# print(eps_to_print_as_table)

eps_to_print_as_table

/tmp/ipykernel_5470/3198637920.py:21: RuntimeWarning: invalid value encountered in divide
  sigmas_list = np.round((c12/c6)**(1/6), decimals=2)
/tmp/ipykernel_5470/3198637920.py:22: RuntimeWarning: invalid value encountered in divide
  epsilons_list = np.round((c6**2)/(4*c12), decimals=2)


38
['PP5', 'PP4', 'PP1', 'PNa', 'PNda', 'PN0', 'PSP1', 'PSNd', 'ARP', 'APP5']
PNa Q0 4.22 3.2
PNa Qa 4.22 3.6
PNa PP5 4.42 4.7
PNa PNa 3.42 3.2
['AR', 'Qd', 'Qa', 'SC3', 'Q0', 'C1', 'SP1', 'PNa', 'PP5', 'C3', 'SC1']


/tmp/ipykernel_5470/2594662346.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  eps_to_print_as_table = pd.DataFrame({bead: pd.Series([]) for bead in list_of_beads})
/tmp/ipykernel_5470/2594662346.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sig_to_print_as_table = pd.DataFrame({bead: pd.Series([]) for bead in list_of_beads})


,AR,Qd,Qa,SC3,Q0,C1,SP1,PNa,PP5,C3,SC1
AR,2.630,2.300,2.300,2.630,2.300,3.500,2.030,2.700,2.000,3.500,2.630
Qd,2.300,3.500,4.000,2.700,2.300,2.300,5.600,5.320,5.320,2.700,2.300
Qa,2.300,4.000,3.500,2.700,2.300,2.300,5.600,3.600,5.320,2.700,2.300
SC3,2.630,2.700,2.700,2.630,2.700,3.500,2.630,2.700,2.700,3.500,2.630
Q0,2.300,2.300,2.300,2.700,3.500,2.300,4.500,3.200,5.320,2.700,2.300
C1,3.500,2.300,2.300,3.500,2.300,3.500,2.700,2.700,2.000,3.500,3.500
SP1,2.030,5.600,5.600,2.630,4.500,2.700,3.370,4.500,5.600,3.500,2.030
PNa,2.700,5.320,3.600,2.700,3.200,2.700,4.500,3.200,4.700,2.700,2.700
PP5,2.000,5.320,5.320,2.700,5.320,2.000,5.600,4.700,5.020,2.700,2.000
C3,3.500,2.700,2.700,3.500,2.700,3.500,3.500,2.700,2.700,3.500,3.500


In [20]:
type(eps_to_print_as_table)

pandas.core.frame.DataFrame

In [70]:
eps_to_print_as_table.to_csv('new_ff_pc_chol_abeta.csv')

In [63]:
x.loc[('D1', 'Dvs')]

c6                        0
c12                     0.0
sigma      ; sigma infinity
epsilon                    
Name: (D1, Dvs), dtype: object

In [ ]:
ff